In [46]:
import pandas as pd
import json
from pprint import pprint
from collections import defaultdict

In [47]:
with open('./states_2010.json') as f:
    data = json.load(f)

In [48]:
csv_data = pd.read_csv(
    './StateData.csv',
    error_bad_lines=False
)
df = pd.DataFrame(csv_data)

In [49]:
# Note geojson file includes DC & Puerto Rico
dataset = data['features']
len(dataset)

52

In [50]:
# state_names = []
errors = []
# for state in dataset:
#     state_names.append(state['properties']['NAME'].lower())
state_code_key = defaultdict()

all_correct_total_exp = []
all_correct_by_total = []
all_correct_per_capita = []

# Refactor this for loop with a helper function to take a single column name (instead of writing the column name 4 times each time)

for state in dataset:
    try:
        df_state = df[(df['Name'].str.lower() == state['properties']['NAME'].lower()) & (df['Year4'] == 2016)]
#         print(state['properties']['STATE'])
#         print(int(df_state['State_Code']))
        state_code_key[int(state['properties']['STATE'])] = int(df_state['State_Code'])
        population = int(df_state.iloc[0]['Population'])
        total_revenue = int(df_state.iloc[0]['Total_Revenue'])
        total_expenditure = int(df_state.iloc[0]['Total_Expenditure'])
        correct_total_exp = int(df_state.iloc[0]['Correct_Total_Exp'])
        all_correct_total_exp.append(correct_total_exp)
        all_correct_by_total.append(correct_total_exp/total_expenditure)
        all_correct_per_capita.append(correct_total_exp/population)
        state['properties']['population'] = population
        state['properties']['total_revenue'] = total_revenue
        state['properties']['total_expenditure'] = total_expenditure
        state['properties']['correct_total_exp'] = correct_total_exp
        state['properties']['correct_as_fraction_of_total_exp'] = correct_total_exp/total_expenditure
        
        state['properties']['correct_per_capita'] = correct_total_exp/population
        
    except:
        errors.append((df['Name'].str.lower()))
                      
                      
                      

In [45]:
dict(state_code_key)

{4: 3,
 5: 4,
 6: 5,
 8: 6,
 9: 7,
 13: 11,
 15: 12,
 17: 14,
 18: 15,
 22: 19,
 27: 24,
 28: 25,
 30: 27,
 35: 32,
 38: 35,
 40: 37,
 42: 39,
 47: 43,
 51: 47,
 10: 8,
 54: 49,
 55: 50,
 56: 51,
 1: 1,
 2: 2,
 12: 10,
 16: 13,
 20: 17,
 24: 21,
 34: 31,
 37: 34,
 45: 41,
 53: 48,
 50: 46,
 49: 45,
 19: 16,
 21: 18,
 23: 20,
 25: 22,
 26: 23,
 29: 26,
 31: 28,
 32: 29,
 33: 30,
 36: 33,
 39: 36,
 41: 38,
 44: 40,
 46: 42,
 48: 44}

In [13]:
max(all_correct_per_capita)


0.4502012372161553

In [14]:
min(all_correct_per_capita)

0.08212717034493404

In [ ]:
# Max correct exp = 9528475
# Min correct exp = 100955
# Max correct exp by total = .033
# Min correct exp by total = .012
# Max correct exp per capita = .45
# Min correct exp per capita = .08

In [48]:
errors

['puerto rico']

In [51]:
data['features'] = dataset
with open('./modified_states.json', 'w') as outfile:
    json.dump(data, outfile)